## Data Profiling

### About the dataset:

TFI has provided a dataset with 137 restaurants in the training set, and a test set of 100000 restaurants. The data columns include the open date, location, city type, and three categories of obfuscated data: Demographic data, Real estate data, and Commercial data. The revenue column indicates a (transformed) revenue of the restaurant in a given year and is the target of predictive analysis. 

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

# read test file
test_data = pd.read_csv('../input/restaurant-revenue-prediction/test.csv.zip')
# read training data
raw_df = pd.read_csv('../input/restaurant-revenue-prediction/train.csv.zip')

In [ ]:
print(f"Shape of training data: {raw_df.shape}\nShape of test dataset: {test_data.shape} \
    \nFeatures available: {raw_df.columns}")

In [ ]:
# To see all the columns in output this can be done.
pd.options.display.max_columns=None
# To see all rows change max_columns with max_rows

raw_df.head()

In [ ]:
raw_df.isnull().sum().any()

* There are no missing values.
* Last column 'revenue' is our target column.
* Features from P1 to P37 are all numerical features about which we know nothing specifically. So, I will directly feed them into my model.
* Id column is redundant, I will drop it. To avoid it we can choose it as index column while reading data.i.e., using pd.read_csv('*filepath*',index_col=*col_name_or_positional_no*)
* In feature Open date, I will focus on month and year and drop date values.
* City, City Group and Type are categorical columns. To feed them to ML model they need to be converted into machine-readable form which is numerical form.

In [ ]:
raw_df.drop('Id',axis=1,inplace=True)

In [ ]:
sns.distplot(raw_df['revenue'],hist=False)
plt.title('Distribution of Target variable')
sns.despine(); #to remove top and right spines

If we remove outliers our target variable will follow a normal distribution.(with a little bit skew)

In [ ]:
raw_df = raw_df[raw_df['revenue']<8e+06].copy()

In [ ]:
fig,ax = plt.subplots(1,2,figsize=(9,5))
sns.set_style('darkgrid')
sns.countplot(raw_df.Type,ax=ax[0])
ax[0].set_title('Train set')
sns.countplot(test_data.Type,ax=ax[1])
ax[1].set_title('Test set');

:(

There is not even asingle observation of type 'MB' in training set. This is a problem for our model.

In [ ]:
#Create a checkpoint so that we can easily access original dataset
df = pd.concat([raw_df,test_data],axis=0)

# Extracting month and year from date column
df['Open Date'] = pd.to_datetime(df['Open Date'])
df['launch_Month'] = [x.month for x in df['Open Date']]
df['launch_year'] = [x.year for x in df['Open Date']]
df.drop(['Id','Open Date'],axis=1,inplace=True)

In [ ]:
sns.countplot(df['launch_Month'])
plt.title('Month-wise no of launches');

Second half of the year seems to witness more restaurant launches.

In [ ]:
plt.figure(figsize=(15,6))
sns.countplot(df['launch_year'],order=[1996,1997,1998,1999,2000,2002,2004,2005,2006,
                                       2007,2008,2009,2010,2011,2012,2013,2014])
plt.title('Year-wise no of launches',fontsize=15);

With time, number of restaurants in the area is increasing due to several social factors.

## Preprocessing the data

In [ ]:
df['Type'].value_counts()

In [ ]:
df['City Group'].value_counts()

In [ ]:
df['City'].value_counts()

**Oops!** There are 63 different City values. I can create dummy variables to handle this feature but it will lead to so many columns. A good approach will be to reduce categories in this column. For example, instead of cities categories can be different zones or tier-1,tier-2 and so on.
Here, I am dropping this column as there is not much information about which countries are involved. Also, the feature City Group covers effect of this feature as well.

In [ ]:
df.drop('City',axis=1,inplace=True)

In [ ]:
#converting other categorical columns
df['Type'] = df['Type'].map({'FC':0,'IL':1,'DT':2,'MB':3})

from sklearn.preprocessing import OrdinalEncoder
encoder = OrdinalEncoder()
df['City Group'] = encoder.fit_transform(np.array(df['City Group']).reshape(-1,1))
df['City Group'] = df['City Group'].apply(int)

In [ ]:
df.dropna().groupby('launch_Month')['revenue'].mean()

Revenue is a bit higher in months of Jan, Sept and Oct.

In [ ]:
df.groupby('launch_year')['revenue'].mean()

* Year 1999 has recorded highest revenues.
* Years 2000, 2013 and 2014 proved to be worst years for restaurants. Though number of observations is too low for year 2014, may be the big picture is differnt from this

In [ ]:
# creating dummy variables
df.launch_year = df.launch_year.astype(str)
df.launch_Month = df.launch_Month.astype(str)
year_dummy = pd.get_dummies(df[['launch_year','launch_Month']],drop_first=True)
df = pd.concat([df,year_dummy],axis=1)
df.drop(['launch_year','launch_Month'],axis=1, inplace=True)

In [ ]:
# Re-splitting train and test data
processed_df = df.dropna(axis=0)
processed_test_data = df[128:].drop('revenue',axis=1)
# remember there were 137 rows in train data

In [ ]:
processed_df.shape,processed_test_data.shape

In [ ]:
#Check it once
processed_df.head()

## Modelling

Split train data further into training and testing sets to see performance of each ML model.
Then, apply models.

In [ ]:
from sklearn.model_selection import train_test_split
X=processed_df.drop('revenue',axis=1)
y=df['revenue'][:128]
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.20,random_state=12345)

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error,mean_squared_error,r2_score
from sklearn.model_selection import cross_val_score

regressors = {
    'Linear Regression' : LinearRegression(),
    'Logistic Regression' : LogisticRegression(),
    'Decision Tree' : DecisionTreeRegressor(),
    'Random Forest' : RandomForestRegressor(),
    'Support Vector Machines' : SVR(),
    'K-nearest Neighbors' : KNeighborsRegressor(),
    'XGBoost' : XGBRegressor()
}
results=pd.DataFrame(columns=['MAE','MSE','R2-score'])
for method,func in regressors.items():
    func.fit(X_train,y_train)
    pred = func.predict(X_test)
    results.loc[method]= [mean_absolute_error(y_test,pred),
                          mean_squared_error(y_test,pred),
                          r2_score(y_test,pred)
                         ]
results

In [ ]:
from sklearn.model_selection import GridSearchCV
parameters = {'learning_rate': [.03, 0.05, .07,.09], #so called `eta` value
              'max_depth': [6,7,8,9],
              'min_child_weight': [4],
              'subsample': [0.7],
              'colsample_bytree': [0.7],
              'n_estimators': [500,700]}

xgb_grid = GridSearchCV(XGBRegressor(),
                        parameters,
                        cv = 3,
                        n_jobs = 5,
                        verbose=True)

xgb_grid.fit(X,y)

print(xgb_grid.best_score_)
print(xgb_grid.best_params_)

In [ ]:
xgb=XGBRegressor(colsample_bytree=0.7,learning_rate=0.05,max_depth=7,min_child_weight=4,
                n_estimators=500,subsample=0.7)
xgb.fit(X,y)
predicted_test_values = xgb.predict(processed_test_data)
submission1 = pd.DataFrame(columns=['Id','Prediction'])
submission1['Id'] = test_data['Id']
submission1['Prediction'] = predicted_test_values
submission1.to_csv('submission_xgb.csv',index=False)

In [ ]:
knn=KNeighborsRegressor(n_neighbors=5)
knn.fit(X_train,y_train)
predicted_test_values2 = knn.predict(processed_test_data)
submission2 = pd.DataFrame(columns=['Id','Prediction'])
submission2['Id'] = test_data['Id']
submission2['Prediction'] = predicted_test_values2
submission2.to_csv('submission_knn.csv',index=False)